# Chapter 5 Tutorial: Making a language model more helpful with RLHF

## Installation and imports

In [ ]:
%pip install -q accelerate -U

In [ ]:
%load_ext autoreload
%autoreload 2

%pip install -q transformers trl evaluate

In [ ]:
## Log into Huggingface hub.

import os
from huggingface_hub import notebook_login
if not os.path.exists('/root/.cache/huggingface/token'):
    notebook_login()

### Import dependencies

In [ ]:
import torch
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from IPython.display import Image, display_png, clear_output
import numpy as np

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

from evaluate import load

In [ ]:
## Configure the PPO model

config = PPOConfig(
    model_name="aisquared/dlite-v1-355m",
    mini_batch_size=16,
    batch_size=16
)

## Load and format data

The dataset used in this tutorial was made available by Anthropic. It consists of manually written prompts representing human inputs, with two possible answers coming from a virtual assistant. The two answers have been reviewed by humans, and in each case one was chosen as being more helpful than the other.

In [ ]:
ds = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base", split="train")
print(ds)

## Create prompts by chopping off the Assistant response in the dataset
new_col = [x[len("\n\nHuman: "):x.find("Assistant:")]
           for x in ds["chosen"]]
new_col = [x.replace("\n", " ").strip() for x in new_col]
ds = ds.add_column("instruction", new_col)
ds = ds.filter(lambda x: len(x["instruction"]) < 100)
ds = ds.select(range(6400))

## Print two examples
examples = ds.select((0, 1))
print(examples["instruction"])

The DLite model uses special tokens, so we will need to format our prompts accordingly.

In [ ]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request."
)

## This is the prompt that is used for generating responses using an already trained model.  It ends with the response
## key, where the job of the model is to provide the completion that follows it (i.e. the response itself).
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro='',
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

In [ ]:
## Print templatized versions of the two examples
ds = ds.add_column('query', [PROMPT_FOR_GENERATION_FORMAT.format(instruction=i) for i in ds['instruction']])
ds.select((0, 1))['query']

### Tokenize the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["query"])
    sample["query"] = sample['instruction']
    return sample

dataset = ds.map(tokenize, batched=False)
dataset.set_format(type="torch")
dataset.shape

In [ ]:
dataset[0]

## Load models

The RLHF process begins with an existing pre-trained model. Here we are using DLite, since it is relatively small and can be fine-tuned with limited GPU usage. A larger model would generate significantly better responses, but the reinforcement learning mechanics still operate in much the same way. We also download a reward model, reward-model-deberta-v3-large-v2.

### Pre-trained DLite model

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

### Reward model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

reward_name = "OpenAssistant/reward-model-deberta-v3-large-v2"
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_name)
reward_tokenizer =  AutoTokenizer.from_pretrained(reward_name)

## Test with a question, and good and bad answers. The better answer should
## produce a larger reward.

question = "How do I teach kids to meditate?"
helpful = "I'm glad you want to teach your kids about meditation!"
bad = "I am not able to answer this question."

inputs = reward_tokenizer(question, helpful, return_tensors='pt')
good_score = reward_model(**inputs).logits[0].cpu().detach()

inputs = reward_tokenizer(question, bad, return_tensors='pt')
bad_score = reward_model(**inputs).logits[0].cpu().detach()
print(good_score, bad_score)


The example above shows that the reward model is giving a higher score to a more helpful response, so our hope is that we can train a more helpful model by encouraging it to generate responses with higher rewards.

## Training the model

Now it's time to do reinforcement learning. We'll run PPO using the reward model as the basis for the reward function. The PPO trainer is initialized with two copies of the generative LLM. One will remain frozen for use as a reference, while the other will be the initial policy that's iteratively trained with PPO. The resulting policy after training is a new version of the generated model with the weights optimized for better human ratings. In each iteration of the training loop, the following steps occur:
1. A batch of prompts is passed through both the policy LLM and the frozen copy.
2. The responses from the policy are fed into the reward model to score their quality.
3. Gradient descent is used to update the weights with the objective of maximizing the reward. Note that the reward model is typically not an adequate reward function on its own. The construction of the reward function is an active area of research, but it almost always includes KL divergence between the updated policy and the original reference LLM. The goal is to ensure that the policy doesn't overfit to the reward model and forget too much information that it had previously learned.

In [ ]:
gen_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": tokenizer.encode(END_KEY)
}

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset,
                         data_collator=collator)

In [ ]:
use_cuda = torch.cuda.is_available()

device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if use_cuda else "cpu"

if use_cuda:
    reward_model = reward_model.cuda()

In [ ]:
%matplotlib notebook

## Initialize notebook
plt.ion()
fig, ax = plt.subplots()
xs = []
moving_avg = []

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from dlite
    response_tensors = []
    for query in query_tensors:
        gen_len = 16
        response = ppo_trainer.generate(query, max_new_tokens=gen_len, **gen_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    ## Calculate the rewards from the reward model
    inputs = reward_tokenizer(batch["query"], batch["response"], return_tensors="pt", padding=True)
    inputs = inputs.to(device)
    rewards = reward_model(**inputs).logits.cpu().detach()
    rewards = [r[0] for r in rewards]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    xs.append(stats["ppo/mean_scores"])

    if epoch < 9:
        continue

    moving_avg.append(np.mean(xs[-10:]))

    ## Print out performance by step
    ax.clear()
    ax.plot(range(10, epoch+2), moving_avg)

    ax.set_xlabel("Number of batches trained")
    ax.set_ylabel("Mean reward over last 10 batches")
    io = BytesIO()
    fig.savefig(io, format="png")

    clear_output(wait=True)
    display_png(Image(io.getvalue()))

    print(batch["query"][0], batch["response"][0])


In [ ]:
## Push model to the Hugging Face hub for easier evaluation

## Set a link to push the model too, starting with your HF account name, followed
## by a slash, and then by whatever name you choose.
MODEL_NAME = 'your-hf-username/your-model-name'

model.push_to_hub(MODEL_NAME)
tokenizer.push_to_hub(MODEL_NAME)

## Model inspection

In [ ]:
## Get a batch from the dataset
bs = 16
test_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
test_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

## Get response from both original and new dlite
for i in range(bs):
    gen_len = 16
    # gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

## Decode responses
test_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
test_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

inputs = reward_tokenizer(test_data['query'], test_data['response (before)'], return_tensors='pt', padding=True)
inputs = inputs.to(device)
test_data["rewards (before)"] = [r[0] for r in reward_model(**inputs).logits.cpu().detach()]

inputs = reward_tokenizer(test_data['query'], test_data['response (after)'], return_tensors='pt', padding=True)
inputs = inputs.to(device)
test_data["rewards (after)"] = [r[0] for r in reward_model(**inputs).logits.cpu().detach()]


In [ ]:
## Store results in a dataframe
df_results = pd.DataFrame(test_data)
df_results

In [ ]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

In [ ]:
for i, row in df_results.sort_values('rewards (after)', ascending=False)[:5].iterrows():
  print('query:', row['query'])
  print('response (before):', row['response (before)'])
  print('response (after):', row['response (after)'])
  print()

### Calculate perplexity
Note that you may need to restart your instance at this time to free memory for the calculation, and between the two calculations below.

In [ ]:
MODEL_NAME = 'your-hf-username/your-model-name'

In [ ]:
test_ds = load_dataset('Anthropic/hh-rlhf', data_dir="helpful-base", split="test")
test_ds

In [ ]:
perplexity = load("perplexity", module_type="metric")
results = perplexity.compute(predictions=test_ds['chosen'],
                             model_id=config.model_name)

In [ ]:
## Perpelxity measurement for the original dlite model
results['mean_perplexity']

In [ ]:
perplexity = load("perplexity", module_type="metric")
results = perplexity.compute(predictions=test_ds['chosen'],
                             model_id=MODEL_NAME)

In [ ]:
## Perpelxity measurement for the RLHF-tuned dlite model
results['mean_perplexity']